In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import sys


from bokeh.io import show, output_file, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, StaticLayoutProvider
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4, Spectral8, Spectral11
from scipy.io import loadmat

output_notebook()
%matplotlib inline

In [ ]:
connectivity = loadmat("../data/aux_meanACS69.mat")['meanACS']
region_names = pd.read_csv('../data/dk_names.csv')['Atlas']
coords = pd.read_csv('../data/dk.csv').set_index('Name')

conn = pd.DataFrame(connectivity)
conn.columns = list(region_names)
conn.index = list(region_names)

In [ ]:
for region in region_names:
    if region not in coords.index:
        conn = conn.drop(index=region, columns=region)

# clip out any connectivity below cutoff (e.g. 2)
cutoff = 2.
conn = conn.clip(lower=cutoff).replace(cutoff, 0)

coords = coords.sort_values(by=['hemisphere','lobe'])

In [ ]:
# assign coordinates to each region, first in the top, then bottom. Always from (0,0)
radius = 1.0
positions = {}
shift = 2*np.pi/136
for i in range(68):
    region = coords.index[i]
    if i < 34:
        x = radius*np.cos(2*np.pi*i/68 + np.pi/2. + shift)
        y = radius*np.sin(2*np.pi*i/68 + np.pi/2. + shift)
    else:
        i = i - 33
        x = radius*np.cos(-2*np.pi*i/68 + np.pi/2. + shift)
        y = radius*np.sin(-2*np.pi*i/68 + np.pi/2. + shift)
    positions.update({region:[x,y]})

In [ ]:
def calculate_color_list(regions_list):
    # create color dictionary
    color_list = []
    ordered_names = regions_list
    for region in ordered_names:
        lobe = coords['lobe'].loc[region]
        lobe_index = list(np.unique(coords.lobe)).index(lobe)
        color = Spectral11[2*lobe_index]
        color_list.append(color)
    return(color_list)

In [ ]:
TOOLTIPS = """
    <div>
        <div>
            <span style="font-size: 8px; font-weight: bold;">@name</span><br>
            <span style="font-size: 8px; ">@n0, @w0</span><br>
            <span style="font-size: 8px; ">@n1, @w1</span><br>
            <span style="font-size: 8px; ">@n2, @w2</span><br>
            <span style="font-size: 8px; ">@n3, @w3</span><br>
            <span style="font-size: 8px; ">@n4, @w4</span><br>
            <span style="font-size: 8px; ">@n5, @w5</span><br>
            <span style="font-size: 8px; ">@n6, @w6</span><br>
            <span style="font-size: 8px; ">@n7, @w7</span><br>
            <span style="font-size: 8px; ">@n8, @w8</span><br>
            <span style="font-size: 8px; ">@n9, @w9</span><br>
            <span style="font-size: 8px; ">@n10, @w10</span><br>
            <span style="font-size: 8px; ">@n11, @w11</span><br>
        </div>
    </div>
"""

In [ ]:
G = nx.from_pandas_adjacency(conn)

plot = Plot(plot_width=600, plot_height=800,
            x_range=Range1d(-1.5,1.5), y_range=Range1d(-1.5,1.5))

plot.title.text = "Brain Connectivity - DK Atlas"

plot.add_tools(HoverTool(tooltips=TOOLTIPS), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0,0))

graph_renderer.node_renderer.glyph = Circle(size=12, fill_color='color')
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color='color')
graph_renderer.node_renderer.hover_glyph = Circle(size=18, fill_color='color')

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=3)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[1], line_width='log_weight')
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[2], line_alpha=0.5, line_width=3)

graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.selection_policy = NodesAndLinkedEdges()

# make positions fixed according to pre-calculated values
fixed_layout_provider = StaticLayoutProvider(graph_layout=positions)
graph_renderer.layout_provider = fixed_layout_provider

color_list = calculate_color_list(graph_renderer.node_renderer.data_source.data['index'])

graph_renderer.node_renderer.data_source.data.update({'name':graph_renderer.node_renderer.data_source.data['index']})

for i in range(12):
    graph_renderer.node_renderer.data_source.data.update({'n'+str(i):neighbors_df[i]})
    graph_renderer.node_renderer.data_source.data.update({'w'+str(i):weights_df[i]})
    

graph_renderer.node_renderer.data_source.data.update({'color':color_list})
graph_renderer.edge_renderer.data_source.data.update({'log_weight':list(np.log(graph_renderer.edge_renderer.data_source.data['weight']))})

plot.renderers.append(graph_renderer)
# output_file("interactive_graphs.html")
output_notebook()
show(plot)


In [ ]:
neighbor_list = []
neighbors_weights = []

for region in graph_renderer.node_renderer.data_source.data['index']:
    neighbor_list.append([n for n in G.neighbors(region)])

for c in range(len(conn.columns)):
    n = list(conn[conn.columns[c]][conn[conn.columns[c]] > 0])
    neighbors_weights.append(n)

neighbors_df = pd.DataFrame(neighbor_list)
weights_df = pd.DataFrame(neighbors_weights)

### todo:
- add lobe legend
- highlight nodes with connection
- add [Bezier](https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html)
- add information: node name and strength of connection
- remove NaN
- Add "threshold" option

### done:
- add name to nodes with connection
- right vs left
- color nodes by lobe
